In [ ]:
pip install xgboost lightgbm catboost sklego optuna

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, train_test_split 
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklego.linear_model import LADRegression
from sklearn.decomposition import PCA

In [11]:
train = pd.read_csv('train.csv').drop(columns = ['id'])
test = pd.read_csv('test.csv').drop(columns = ['id'])
sub = pd.read_csv('sample_submission.csv')
original = pd.read_csv('original.csv')

train = pd.concat([train, original], axis = 0).reset_index(drop = True)

## Rounding

In [ ]:
new = pd.read_csv('submissions/Ensemble_submission.csv')

In [ ]:
new.head()

In [ ]:
new['Age'] = np.round(new['Age']).astype(int)
new.head()

In [ ]:
new.to_csv('submissions/Ensemble_rounded.csv', index = False)

## Exploratory Data Analysis:

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
print(train.isna().sum())
print('\n', test.isna().sum())

In [ ]:
train.drop(columns = ['Sex']).corr()

In [ ]:
fig, axes = plt.subplots(2, 4, figsize = (20, 10))
plt.tight_layout(pad = 5)

sns.kdeplot(ax = axes[0, 0], data = train, x = 'Length', fill = True).set_title('Length')
sns.kdeplot(ax = axes[0, 1], data = train, x = 'Diameter', fill = True).set_title('Diameter')
sns.kdeplot(ax = axes[0, 2], data = train, x = 'Height', fill = True).set_title('Height')
sns.kdeplot(ax = axes[0, 3], data = train, x = 'Weight', fill = True).set_title('Weight')
sns.kdeplot(ax = axes[1, 0], data = train, x = 'Shucked Weight', fill = True).set_title('Shucked Weight')
sns.kdeplot(ax = axes[1, 1], data = train, x = 'Viscera Weight', fill = True).set_title('Viscera Weight')
sns.kdeplot(ax = axes[1, 2], data = train, x = 'Shell Weight', fill = True).set_title('Shell Weight')
sns.kdeplot(ax = axes[1, 3], data = train, x = 'Age', fill = True).set_title('Age')
plt.show()

## Feature Engineering and Cleaning:

In [12]:
## Weight
train['Accounted Weight'] = train['Shucked Weight'] + train['Viscera Weight'] + train['Shell Weight']
train['Weight Diff.'] = train['Weight'] - train['Accounted Weight']
train['Too Heavy'] = np.where(train['Accounted Weight'] > train['Weight'], 1, 0).astype(int)
train['Shucked Weight'] = np.where(train['Accounted Weight'] > train['Weight'], 0.424150 * train['Weight'], train['Shucked Weight'])
train['Viscera Weight'] = np.where(train['Accounted Weight'] > train['Weight'], 0.213569 * train['Weight'], train['Viscera Weight'])
train['Shell Weight'] = np.where(train['Accounted Weight'] > train['Weight'], 0.288712 * train['Weight'], train['Shell Weight'])
train['Shucked Weight Perc.'] = train['Shucked Weight'] / train['Weight']
train['Viscera Weight Perc.'] = train['Viscera Weight'] / train['Weight']
train['Shell Weight Perc.'] = train['Shell Weight'] / train['Weight']

test['Accounted Weight'] = test['Shucked Weight'] + test['Viscera Weight'] + test['Shell Weight']
test['Weight Diff.'] = test['Weight'] - test['Accounted Weight']
test['Too Heavy'] = np.where(test['Accounted Weight'] > test['Weight'], 1, 0).astype(int)
test['Shucked Weight'] = np.where(test['Accounted Weight'] > test['Weight'], 0.424150 * test['Weight'], test['Shucked Weight'])
test['Viscera Weight'] = np.where(test['Accounted Weight'] > test['Weight'], 0.213569 * test['Weight'], test['Viscera Weight'])
test['Shell Weight'] = np.where(test['Accounted Weight'] > test['Weight'], 0.288712 * test['Weight'], test['Shell Weight'])
test['Shucked Weight Perc.'] = test['Shucked Weight'] / test['Weight']
test['Viscera Weight Perc.'] = test['Viscera Weight'] / test['Weight']
test['Shell Weight Perc.'] = test['Shell Weight'] / test['Weight']

In [13]:
## Dimensions
train['Height'] = np.where(train['Height'] > 2, np.mean(train['Height']), 
                           np.where(train['Height'] == 0, 0.29337*train['Length']-0.03826729, train['Height']))
train['Volume'] = train['Length'] * train['Diameter'] * train['Height']
train['Density'] = train['Weight'] / train['Volume']

test['Height'] = np.where(test['Height'] > 2, np.mean(test['Height']), 
                           np.where(test['Height'] == 0, 0.29400666*test['Length']-0.03933592, test['Height']))
test['Volume'] = test['Length'] * test['Diameter'] * test['Height']
test['Density'] = test['Weight'] / test['Volume']

In [14]:
## Gender
train['Male'] = np.where(train['Sex'] == 'M', 1, 0); train['Female'] = np.where(train['Sex'] == 'F', 1, 0)
test['Male'] = np.where(test['Sex'] == 'M', 1, 0); test['Female'] = np.where(test['Sex'] == 'F', 1, 0)

In [15]:
## PCA
numeric_features = ['Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight', 'Viscera Weight', 'Shell Weight', 'Accounted Weight', 
                    'Weight Diff.', 'Shucked Weight Perc.', 'Viscera Weight Perc.', 'Shell Weight Perc.', 'Volume', 'Density']

scaler = StandardScaler().fit(train[numeric_features])
X_train = scaler.transform(train[numeric_features])
X_test = scaler.transform(test[numeric_features])

pca = PCA(4).fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

X_train_pca = pd.DataFrame(X_train_pca, columns = ['PC_1', 'PC_2', 'PC_3', 'PC_4'])
X_test_pca = pd.DataFrame(X_test_pca, columns = ['PC_1', 'PC_2', 'PC_3', 'PC_4'])

train = pd.concat([train, X_train_pca], axis = 1)
test = pd.concat([test, X_test_pca], axis = 1)

In [16]:
train = train.drop(columns = ['Sex', 'Too Heavy'])
test = test.drop(columns = ['Sex', 'Too Heavy'])

## Hyper-parameter Tuning:

In [19]:
## Defining input and target variables
X = train.drop(columns = ['Age'])
Y = train['Age']

## Initializing parameters
SEED = 42

## Defining Optuna objective functions
def RF_objective(trial):

    ## Defining the hyper-parameter grid
    param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1000, 50), 
                  'max_depth': trial.suggest_int('max_depth', 3, 12),  
                  'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),  
                  'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),  
                  'random_state': trial.suggest_int('random_state', 1, 500), 
                  'max_features': trial.suggest_categorical('max_features', ['sqrt', None])
                 }
    scores = list()
    kf = KFold(n_splits = 10, shuffle = True, random_state = SEED)
    
    for train_idx, valid_idx in kf.split(X, Y):
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        Y_train, Y_valid = Y.iloc[train_idx], Y.iloc[valid_idx]
        
        ## Building the model
        model = RandomForestRegressor(**param_grid, n_jobs = -1, criterion = 'absolute_error').fit(X_train, Y_train)
        
        ## Predicting on the test data-frame
        preds = model.predict(X_valid)
        
        ## Evaluating model performance on the test set
        scores.append(mean_absolute_error(Y_valid, preds))
    
    return np.mean(scores)

def HIST_objective(trial):

    ## Defining the hyper-parameter grid
    param_grid = {'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01), 
                  'max_iter': trial.suggest_int('n_estimators', 100, 1000, 50), 
                  'max_depth': trial.suggest_int('max_depth', 3, 12),  
                  'l2_regularization': trial.suggest_float('l2_regularization', 0, 0.1, step = 0.002), 
                  'random_state': trial.suggest_int('random_state', 1, 500),
                 }
    scores = list()
    kf = KFold(n_splits = 10, shuffle = True, random_state = SEED)
    
    for train_idx, valid_idx in kf.split(X, Y):
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        Y_train, Y_valid = Y.iloc[train_idx], Y.iloc[valid_idx]
        
        ## Building the model
        model = HistGradientBoostingRegressor(**param_grid, loss = 'absolute_error', early_stopping = True).fit(X_train, Y_train)
        
        ## Predicting on the test data-frame
        preds = model.predict(X_valid)
        
        ## Evaluating model performance on the test set
        scores.append(mean_absolute_error(Y_valid, preds))
    
    return np.mean(scores)

def XGB_objective(trial):

    ## Defining the hyper-parameter grid
    param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1000, 50),  
                  'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01),  
                  'max_depth': trial.suggest_int('max_depth', 3, 12),  
                  'gamma': trial.suggest_float('gamma', 0, 0.3, step = 0.05),  
                  'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),  
                  'subsample': trial.suggest_float('subsample', 0.6, 1, step = 0.05),  
                  'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1, step = 0.05), 
                  'seed': trial.suggest_int('seed', 1, 1000) 
                 }
    scores = list()
    kf = KFold(n_splits = 10, shuffle = True, random_state = SEED)
    
    for train_idx, valid_idx in kf.split(X, Y):
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        Y_train, Y_valid = Y.iloc[train_idx], Y.iloc[valid_idx]
        
        ## Building the model
        model = XGBRegressor(**param_grid, n_jobs = -1, booster = 'gbtree', tree_method = 'hist').fit(X_train, Y_train)
        
        ## Predicting on the test data-frame
        preds = model.predict(X_valid)
        
        ## Evaluating model performance on the test set
        scores.append(mean_absolute_error(Y_valid, preds))
    
    return np.mean(scores)

def LGBM_objective(trial):
    
    ## Defining the hyper-parameter grid
    param_grid = {'n_estimators': trial.suggest_int('n_estimators', 100, 1000, 50), 
                  'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step = 0.01), 
                  'num_leaves': trial.suggest_int('num_leaves', 5, 40, step = 1), 
                  'max_depth': trial.suggest_int('max_depth', 3, 12), 
                  'subsample': trial.suggest_float('subsample', 0.6, 1, step = 0.05),  
                  'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1, step = 0.05), 
                  'random_state': trial.suggest_int('random_state', 1, 1000),
                 }
    scores = list()
    kf = KFold(n_splits = 10, shuffle = True, random_state = SEED)
    
    for train_idx, valid_idx in kf.split(X, Y):
        
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        Y_train, Y_valid = Y.iloc[train_idx], Y.iloc[valid_idx]
        
        ## Building the model
        model = LGBMRegressor(**param_grid, n_jobs = -1, boosting_type = 'dart', verbosity = -1).fit(X_train, Y_train)
        
        ## Predicting on the test data-frame
        preds = model.predict(X_valid)
        
        ## Evaluating model performance on the test set
        scores.append(mean_absolute_error(Y_valid, preds))
    
    return np.mean(scores)



## Starting RandomForest
## ----
# ## Creating a study object and to optimize the home objective function
# study_rf = optuna.create_study(direction = 'minimize')
# study_rf.optimize(RF_objective, n_trials = 5)

## Starting HistGradientBoosting
## ----
## Creating a study object and to optimize the home objective function
study_hist = optuna.create_study(direction = 'minimize')
study_hist.optimize(HIST_objective, n_trials = 5)

## Starting XGBoost
## ----
## Creating a study object and to optimize the home objective function
study_xgb = optuna.create_study(direction = 'minimize')
study_xgb.optimize(XGB_objective, n_trials = 5)

## Starting LightGBM
## ----
## Creating a study object and to optimize the home objective function
study_lgbm = optuna.create_study(direction = 'minimize')
study_lgbm.optimize(LGBM_objective, n_trials = 5)

# ## Printing best hyper-parameter set
# print('Random Forest: \n', study_rf.best_trial.params)
# print(study_rf.best_trial.value)

## Printing best hyper-parameter set
print('HistGB: \n', study_hist.best_trial.params)
print(study_hist.best_trial.value)

## Printing best hyper-parameter set
print('\nXGBoost: \n', study_xgb.best_trial.params)
print(study_xgb.best_trial.value)

## Printing best hyper-parameter set
print('\nLightGBM: \n', study_lgbm.best_trial.params)
print(study_lgbm.best_trial.value)

[I 2023-06-03 17:19:23,982] A new study created in memory with name: no-name-0a016058-35b1-4ac2-a382-00cb4c485ad8
[W 2023-06-03 17:20:31,879] Trial 0 failed with parameters: {'learning_rate': 0.11, 'n_estimators': 450, 'max_depth': 5, 'l2_regularization': 0.028, 'random_state': 29} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_10413/2256051632.py", line 56, in HIST_objective
    model = HistGradientBoostingRegressor(**param_grid, loss = 'absolute_error', early_stopping = True).fit(X_train, Y_train)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/ensemble/_hist_gradient_boosting/gradient_boosting.py", line 710, in fit
    _update_raw_predictions(raw_predictions[:, k], grower, n_threads)
KeyboardInterrupt
[W 2023-06-03 17:20:31,887

KeyboardInterrupt: 

## Modelling:

In [ ]:
xgb_params = {'n_estimators': 1000, 
              'learning_rate': 0.00482382842096919, 
              'booster': 'gbtree', 
              'lambda': 0.000235366507474591, 
              'alpha': 0.0000115977765684837, 
              'subsample': 0.35955930593108, 
              'colsample_bytree': 0.898528184386095, 
              'max_depth': 9, 
              'min_child_weight': 8, 
              'eta': 0.0000784943239744148, 
              'gamma': 1.6661346939401E-07, 
              'grow_policy': 'lossguide', 
              'n_jobs': -1, 
              'objective': 'reg:squarederror',
              'eval_metric': 'mae', 
              'verbosity': 0}

lgb1_params = {'n_estimators': 1000,
               'learning_rate': 0.00659605502010782,
               "reg_alpha": 0.0134568843414818,
               "reg_lambda": 2.38367559632979E-06,
               "num_leaves": 117,
               "colsample_bytree": 0.850706320762174,
               'subsample': 0.691827302225948,
               'subsample_freq': 4,
               'min_child_samples': 33,
               'objective': 'regression_l2',
               'metric': 'mae',
               'boosting_type': 'gbdt'}

cat1_params = {'iterations': 1000,
               'depth': 7,
               'learning_rate': 0.00454306521731278,
               'l2_leaf_reg': 0.113774158297261,
               'random_strength': 0.0179641854849499,
               'od_type': 'IncToDec',
               'od_wait': 50,
               'bootstrap_type': 'Bayesian',
               'grow_policy': 'Lossguide',
               'bagging_temperature': 1.39240858193441,
               'eval_metric': 'MAE',
               'loss_function': 'MAE',
               'verbose': False,
               'allow_writing_files': False}

hist_params = {'loss': 'absolute_error',
               'l2_regularization': 0.0104104133357932,
               'early_stopping': True,
               'learning_rate': 0.00627298859709192,
               'max_iter': 1000,
               'n_iter_no_change': 200,
               'max_depth': 16,
               'max_bins': 255,
               'min_samples_leaf': 54,
               'max_leaf_nodes':57}

gbd_params = {'loss': 'absolute_error',
              'n_estimators': 800,
              'max_depth': 10,
              'learning_rate': 0.01,
              'min_samples_split': 10,
              'min_samples_leaf': 20}

models = {"xgb": XGBRegressor(**xgb_params),
          "lgb": LGBMRegressor(**lgb1_params),
          "cat": CatBoostRegressor(**cat1_params),
          'hgb': HistGradientBoostingRegressor(**hist_params),
          "SVR_rbf": SVR(kernel="rbf", gamma="auto"),
          "RandomForestRegressor": RandomForestRegressor(n_estimators=500, n_jobs=-1),
          "GradientBoostingRegressor": GradientBoostingRegressor(**gbd_params)}

In [ ]:
## Dropping some variables
train.drop(columns = ['Sex'], axis = 1, inplace = True)
test.drop(columns = ['id', 'Sex'], axis = 1, inplace = True)

## Defining the input and target variables
X = train.drop(columns = ['Age'], axis = 1)
Y = train['Age']

## Defining lists to store results
gb_cv_scores, gb_preds = list(), list()
hist_cv_scores, hist_preds = list(), list()
lgb_cv_scores, lgb_preds = list(), list()
xgb_cv_scores, xgb_preds = list(), list()
ens_cv_scores, ens_preds = list(), list()

## Performing KFold cross-validation
skf = KFold(n_splits = 10, random_state = 42, shuffle = True)
    
for i, (train_ix, test_ix) in enumerate(skf.split(X, Y)):
        
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    
    print('---------------------------------------------------------------')
    
    ######################
    ## GradientBoosting ##
    ######################
        
    gb_md = GradientBoostingRegressor(loss = 'absolute_error', n_estimators = 1000, max_depth = 8, learning_rate = 0.01,
                                      min_samples_split = 10, min_samples_leaf = 20).fit(X_train, Y_train) 
    
    gb_pred_1 = gb_md.predict(X_test[X_test['generated'] == 1])
    gb_pred_2 = gb_md.predict(test)
    
    gb_score_fold = mean_absolute_error(Y_test[X_test['generated'] == 1], gb_pred_1)
    gb_cv_scores.append(gb_score_fold)
    gb_preds.append(gb_pred_2)
    
    print('Fold', i, '==> GradientBoositng oof MAE is ==>', gb_score_fold)
    
    
    ##########################
    ## HistGradientBoosting ##
    ##########################
        
    hist_md = HistGradientBoostingRegressor(loss = 'absolute_error', l2_regularization = 0.01, early_stopping = False, learning_rate = 0.01,
                                            max_iter = 1000, max_depth = 15, max_bins = 255, min_samples_leaf = 30, 
                                            max_leaf_nodes = 30).fit(X_train, Y_train)
    
    hist_pred_1 = hist_md.predict(X_test[X_test['generated'] == 1])
    hist_pred_2 = hist_md.predict(test)

    hist_score_fold = mean_absolute_error(Y_test[X_test['generated'] == 1], hist_pred_1)
    hist_cv_scores.append(hist_score_fold)
    hist_preds.append(hist_pred_2)
    
    print('Fold', i, '==> HistGradient oof MAE is ==>', hist_score_fold)
        
    ##############
    ## LightGBM ##
    ##############
        
    lgb_md = LGBMRegressor(objective = 'mae', n_estimators = 1000, max_depth = 10, learning_rate = 0.01, num_leaves = 70, reg_alpha = 3,
                           reg_lambda = 3, subsample = 0.7, colsample_bytree = 0.7).fit(X_train, Y_train)
    
    lgb_pred_1 = lgb_md.predict(X_test[X_test['generated'] == 1])
    lgb_pred_2 = lgb_md.predict(test)

    lgb_score_fold = mean_absolute_error(Y_test[X_test['generated'] == 1], lgb_pred_1)    
    lgb_cv_scores.append(lgb_score_fold)
    lgb_preds.append(lgb_pred_2)
    
    print('Fold', i, '==> LightGBM oof MAE is ==>', lgb_score_fold)
        
    #############
    ## XGBoost ##
    #############
        
    xgb_md = XGBRegressor(objective = 'reg:pseudohubererror', colsample_bytree = 0.7, gamma = 0.8, learning_rate = 0.01, max_depth = 8, 
                          min_child_weight = 20, n_estimators = 1000, subsample = 0.7).fit(X_train, Y_train)
    
    xgb_pred_1 = xgb_md.predict(X_test[X_test['generated'] == 1])
    xgb_pred_2 = xgb_md.predict(test)

    xgb_score_fold = mean_absolute_error(Y_test[X_test['generated'] == 1], xgb_pred_1)    
    xgb_cv_scores.append(xgb_score_fold)
    xgb_preds.append(xgb_pred_2)
    
    print('Fold', i, '==> XGBoost oof MAE is ==>', xgb_score_fold)
    
    ##################
    ## LAD Ensemble ##
    ##################
    
    x = pd.DataFrame({'GBC':gb_pred_1,  'hist': hist_pred_1, 'lgb': lgb_pred_1, 'xgb': xgb_pred_1})
    y = Y_test[X_test['generated'] == 1]
    
    lad_md = LADRegression().fit(x, y)
    lad_pred = lad_md.predict(x)
    
    x_test = pd.DataFrame({'GBC':gb_pred_2,  'hist': hist_pred_2, 'lgb': lgb_pred_2, 'xgb': xgb_pred_2})
    lad_pred_test = lad_md.predict(x_test)
        
    ens_score = mean_absolute_error(y, lad_pred)
    ens_cv_scores.append(ens_score)
    ens_preds.append(lad_pred_test)
    
    print('Fold', i, '==> LAD ensemble oof MAE is ==>', ens_score)

In [ ]:
print(np.mean(gb_cv_scores))
print(np.mean(hist_cv_scores))
print(np.mean(lgb_cv_scores))
print(np.mean(xgb_cv_scores))
print(np.mean(ens_cv_scores))

In [ ]:
gb_preds_test = pd.DataFrame(gb_preds).apply(np.mean, axis = 0)
hist_preds_test = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)
lgb_preds_test = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
xgb_preds_test = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
ens_preds_test = pd.DataFrame(ens_preds).apply(np.mean, axis = 0)

sub['Age'] = gb_preds_test
sub.to_csv('submissions/GB_submission.csv', index = False)

sub['Age'] = hist_preds_test
sub.to_csv('submissions/Hist_submission.csv', index = False)

sub['Age'] = lgb_preds_test
sub.to_csv('submissions/LightGBM_submission.csv', index = False)

sub['Age'] = xgb_preds_test
sub.to_csv('submissions/XGBoost_submission.csv', index = False)

sub['Age'] = ens_preds_test
sub.to_csv('submissions/Ensemble_submission.csv', index = False)